## Importing Libraries

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Reading Training Data

In [16]:
file_path = r"C:\Projects\AWS-SageMaker-Flight_Price_Predictor\data\train.csv"

train = pd.read_csv(file_path)
train.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-03,Delhi,Cochin,13:15:00,09:25:00,1210,2.0,No Info,10467
1,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,04:25:00,550,1.0,In-flight meal not included,12898
2,Jet Airways,2019-05-21,Kolkata,Banglore,06:30:00,19:50:00,800,1.0,No Info,14781
3,Jet Airways,2019-05-18,Delhi,Cochin,11:00:00,14:15:00,195,0.0,No Info,9564
4,Air India,2019-05-15,Kolkata,Banglore,14:15:00,16:45:00,150,0.0,No Info,5510


In [17]:
train.dtypes

airline             object
date_of_journey     object
source              object
destination         object
dep_time            object
arrival_time        object
duration             int64
total_stops        float64
additional_info     object
price                int64
dtype: object

In [20]:
train = train.assign(**{
    col : pd.to_datetime(train.loc[:,col],dayfirst=True,format="mixed")
    for col in ["date_of_journey","dep_time","arrival_time"]
})
train.dtypes

airline                    object
date_of_journey    datetime64[ns]
source                     object
destination                object
dep_time           datetime64[ns]
arrival_time       datetime64[ns]
duration                    int64
total_stops               float64
additional_info            object
price                       int64
dtype: object

In [21]:
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-06-03,Delhi,Cochin,2024-07-05 13:15:00,2024-07-05 09:25:00,1210,2.0,No Info,10467
1,Jet Airways,2019-05-27,Delhi,Cochin,2024-07-05 19:15:00,2024-07-05 04:25:00,550,1.0,In-flight meal not included,12898
2,Jet Airways,2019-05-21,Kolkata,Banglore,2024-07-05 06:30:00,2024-07-05 19:50:00,800,1.0,No Info,14781
3,Jet Airways,2019-05-18,Delhi,Cochin,2024-07-05 11:00:00,2024-07-05 14:15:00,195,0.0,No Info,9564
4,Air India,2019-05-15,Kolkata,Banglore,2024-07-05 14:15:00,2024-07-05 16:45:00,150,0.0,No Info,5510
...,...,...,...,...,...,...,...,...,...,...
6690,Spicejet,2019-06-24,Kolkata,Banglore,2024-07-05 17:10:00,2024-07-05 19:40:00,150,0.0,No Info,3873
6691,Multiple Carriers,2019-06-15,Delhi,Cochin,2024-07-05 10:20:00,2024-07-05 19:15:00,535,1.0,No Info,9526
6692,Indigo,2019-04-21,Delhi,Cochin,2024-07-05 02:00:00,2024-07-05 07:45:00,345,1.0,No Info,6258
6693,Jet Airways,2019-03-24,Kolkata,Banglore,2024-07-05 21:10:00,2024-07-05 08:10:00,660,1.0,In-flight meal not included,11134


## High Level Summary of data

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6695 entries, 0 to 6694
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          6695 non-null   object        
 1   date_of_journey  6695 non-null   datetime64[ns]
 2   source           6695 non-null   object        
 3   destination      6695 non-null   object        
 4   dep_time         6695 non-null   datetime64[ns]
 5   arrival_time     6695 non-null   datetime64[ns]
 6   duration         6695 non-null   int64         
 7   total_stops      6695 non-null   float64       
 8   additional_info  6695 non-null   object        
 9   price            6695 non-null   int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(4)
memory usage: 523.2+ KB


In [24]:
train.describe(include="number")

,duration,total_stops,price
count,6695.000000,6695.000000,6695.000000
mean,631.109037,0.802390,9047.110829
std,501.727303,0.659055,4565.426652
min,75.000000,0.000000,1759.000000
25%,170.000000,0.000000,5228.000000
50%,505.000000,1.000000,8309.000000
75%,915.000000,1.000000,12351.000000
max,2860.000000,4.000000,57209.000000


In [25]:
(
    train
    .assign(total_stops=train["total_stops"].astype(object))
    .describe(include="O")
)

,airline,source,destination,total_stops,additional_info
count,6695,6695,6695,6695.0,6695
unique,9,5,6,5.0,9
top,Jet Airways,Delhi,Cochin,1.0,No Info
freq,2394,2791,2791,3633.0,5216
